In [5]:
import os
# Definindo a variável de ambiente do SPARK_VERSION com versão do Spark que estou usando:
os.environ["SPARK_VERSION"]="3.4.1"

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructField,
    StructType,
    StringType,
    TimestampType,
    DoubleType,
    IntegerType,
)

SparkSession:

In [7]:
spark = SparkSession.builder.appName('question_4').getOrCreate()

23/09/23 10:38:57 WARN Utils: Your hostname, guerrlr0-A320M-S2H resolves to a loopback address: 127.0.1.1; using 192.168.11.7 instead (on interface enp6s0)
23/09/23 10:38:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/23 10:38:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Schema:

In [14]:
schema = StructType([
    StructField('id', StringType()),
    StructField('data_emissao', TimestampType()),
    StructField('data_vencimento', TimestampType()),
    StructField('valor', DoubleType()),
    StructField('status', StringType()),
    StructField('pagamento_data', TimestampType()),
    StructField('baixa_data', TimestampType()),
    StructField('data_cancelamento', TimestampType()),
    StructField('valor_cancelado', DoubleType()),
    StructField('pagamento_valor', DoubleType()),
    StructField('id_renegociacao', StringType()),
    StructField('id_renegociacao_novo', StringType()),
])

### DataFrame:

In [8]:
path = r'../data/Tabela financeiro - teste Python pleno.csv'

In [23]:
df = spark.read.csv(
    path,
    sep=',',
    header=True,
    schema=schema
).createOrReplaceTempView("base_de_dados")

In [24]:
spark.sql('select * from base_de_dados').show()


+------+-------------------+-------------------+-----+------+-------------------+-------------------+-------------------+---------------+---------------+---------------+--------------------+
|    id|       data_emissao|    data_vencimento|valor|status|     pagamento_data|         baixa_data|  data_cancelamento|valor_cancelado|pagamento_valor|id_renegociacao|id_renegociacao_novo|
+------+-------------------+-------------------+-----+------+-------------------+-------------------+-------------------+---------------+---------------+---------------+--------------------+
| 86794|2019-05-20 00:00:00|2020-06-01 00:00:00| 99.9|     R|2020-05-25 00:00:00|2020-05-26 08:53:20|               null|            0.0|           99.9|           null|                null|
| 89892|2019-05-21 00:00:00|2020-06-01 00:00:00| 99.9|     C|               null|               null|2019-05-21 00:00:00|           99.9|           null|           null|                null|
| 91689|2019-05-22 00:00:00|2020-06-01 00:00:

### Entendimento dos Dados:

Quais são os nomes das colunas?

In [10]:
df.schema.fieldNames()

['id',
 'data_emissao',
 'data_vencimento',
 'valor',
 'status',
 'pagamento_data',
 'baixa_data',
 'data_cancelamento',
 'valor_cancelado',
 'pagamento_valor',
 'id_renegociacao',
 'id_renegociacao_novo']

Quais são os tipos?

In [11]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- data_emissao: string (nullable = true)
 |-- data_vencimento: string (nullable = true)
 |-- valor: string (nullable = true)
 |-- status: string (nullable = true)
 |-- pagamento_data: string (nullable = true)
 |-- baixa_data: string (nullable = true)
 |-- data_cancelamento: string (nullable = true)
 |-- valor_cancelado: string (nullable = true)
 |-- pagamento_valor: string (nullable = true)
 |-- id_renegociacao: string (nullable = true)
 |-- id_renegociacao_novo: string (nullable = true)



### Análise:

a) Qual é o valor total de títulos emitidos no mês de junho de 2020? Qual é o valor total de títulos com vencimento no mesmo período? Por que esses números são diferentes?

Valor total de títulos emitidos em julho (mês 6) de 2020:

In [53]:
query = """
select
    sum(valor)
from base_de_dados
where (
    year(data_emissao) = 2020
    and month(data_emissao) = 6.
)
"""
spark.sql(query).show()

+-----------------+
|       sum(valor)|
+-----------------+
|540360.0000000052|
+-----------------+



Valor total de títulos vencimentos em julho (mês 6) de 2020:

In [54]:
query = """
select
    sum(valor)
from base_de_dados

where (
    year(data_vencimento) = 2020
    and month(data_vencimento) = 6
)
"""
spark.sql(query).show()

+-----------------+
|       sum(valor)|
+-----------------+
|3076802.819998371|
+-----------------+



Quantidade de títulos emitidos em julho (mês 6) de 2020:

In [55]:
query = """
select
    count(data_emissao)
from base_de_dados

where (
    year(data_emissao) = 2020
    and month(data_emissao) = 6
)
"""
spark.sql(query).show()

+-------------------+
|count(data_emissao)|
+-------------------+
|               5423|
+-------------------+



Quantidade de títulos vencimento em julho (mês 6) de 2020:

In [68]:
query = """
select
    count(data_vencimento)
from base_de_dados

where (
    year(data_vencimento) = 2020
    and month(data_vencimento) = 6
)
"""
spark.sql(query).show()

+----------------------+
|count(data_vencimento)|
+----------------------+
|                 32082|
+----------------------+



*O motivo é que havia mais vencimentos agendados para julho (mês 6) de 2020 do que a emissão de boletos em julho (mês 6) de 2020.*

b) Gerar um relatório com todos os clientes que tiveram títulos pagos em julho de 2020.

In [43]:
query = """
select
    id,
    data_emissao,
    data_vencimento,
    valor,
    status,
    pagamento_data,
    baixa_data,
    pagamento_valor
from base_de_dados

where (
    year(pagamento_data) = 2020
    and month(pagamento_data) = 6
)
"""
spark.sql(query).show()

+------+-------------------+-------------------+-----+------+-------------------+-------------------+---------------+
|    id|       data_emissao|    data_vencimento|valor|status|     pagamento_data|         baixa_data|pagamento_valor|
+------+-------------------+-------------------+-----+------+-------------------+-------------------+---------------+
| 95256|2019-05-24 00:00:00|2020-06-01 00:00:00| 79.9|     R|2020-06-01 00:00:00|2020-06-02 09:17:42|           79.9|
| 99775|2019-05-28 00:00:00|2020-06-01 00:00:00| 79.9|     R|2020-06-02 00:00:00|2020-06-03 10:42:59|          81.62|
|100545|2019-05-28 00:00:00|2020-06-01 00:00:00| 79.9|     R|2020-06-30 00:00:00|2020-07-01 08:54:37|          85.26|
|101580|2019-05-29 00:00:00|2020-06-01 00:00:00| 69.9|     R|2020-06-04 00:00:00|2020-06-05 09:41:29|          71.62|
|104833|2019-05-31 00:00:00|2020-06-01 00:00:00| 69.9|     R|2020-06-09 00:00:00|2020-06-10 09:32:15|          72.18|
|106498|2019-06-03 00:00:00|2020-06-01 00:00:00| 64.9|  

c) Encontre quantos boletos foram renegociados no mês de julho de 2020.

In [79]:
query = """
select
    count(id)
from base_de_dados

where (
    year(data_emissao) = 2020
    and month(data_emissao) = 6
    and (
        id_renegociacao is not null
        or id_renegociacao_novo is not null
    )
)
"""
spark.sql(query).show()

+---------+
|count(id)|
+---------+
|        3|
+---------+



d) Encontre o valor pago de juros de todos os boletos pagos em junho de 2020.

Valor total de juros pagos por boletos em junho de 2020:

In [47]:
query = """
select
    sum(valor) - sum(pagamento_valor) as juros_total
from base_de_dados

where (
    year(data_vencimento) = 2020
    and status = 'R'
)
"""
spark.sql(query).show()

+------------------+
|       juros_total|
+------------------+
|-77888.95000096597|
+------------------+



Juros pagos por boletos em junho de 2020:

In [46]:
query = """
select
    valor - pagamento_valor as juros_total
from base_de_dados

where (
    year(data_vencimento) = 2020
    month()
    and status = 'R'
)
"""
spark.sql(query).show()

+-------------------+
|        juros_total|
+-------------------+
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|-1.7199999999999989|
| -5.359999999999999|
|                0.0|
|-1.7199999999999989|
|                0.0|
| -2.280000000000001|
|                0.0|
|-3.0799999999999983|
| -5.359999999999999|
|-2.1599999999999966|
| -5.519999999999996|
|-10.159999999999997|
|-2.6400000000000006|
| -4.239999999999995|
|                0.0|
+-------------------+
only showing top 20 rows



e) Encontre o id dos boletos que foram usados para gerar uma renegociação em junho de 2020.

In [76]:
query = """
select
    id
from base_de_dados

where (
    year(data_emissao) = 2020
    and month(data_emissao) = 6
    and (
        id_renegociacao is not null
        or id_renegociacao_novo is not null
    )
)
"""
spark.sql(query).show()

+------+
|    id|
+------+
|400729|
|407525|
|407525|
+------+

